In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf 
from tensorflow import keras
from keras import Input
from keras import layers
from keras.optimizers import RMSprop
from keras import backend as K
from scipy.spatial import distance

def ComputeMaxDistance(X1, X2):
    max_dist = 0
    for x1 in X1:
        for x2 in X2:
            dist = distance.euclidean(x1, x2)
            max_dist = max(dist, max_dist)
    return max_dist

def RBF_kernel(inputs, centers, sigma):
  X_ = tf.expand_dims(inputs, axis=1)
  C_ = tf.expand_dims(tf.transpose(centers), axis=0)
  D_ = X_ - C_
  return tf.exp(-tf.norm(D_, axis=2)**2) / (2 * sigma**2) 

class RBF(keras.layers.Layer):

  def __init__(self, num_units):
    self.num_units = num_units
    super(RBF, self).__init__()

  def build(self, input_shape):
    self.centers = self.add_weight(shape=(input_shape[1], self.num_units), initializer=tf.initializers.RandomNormal, trainable=False) # TODO use custom initializer
    self.sigma = self.add_weight(shape=(1, 1), initializer=tf.initializers.RandomNormal, trainable=False) # TODO use custom initializer
    super(RBF, self).build(input_shape)

  def call(self, inputs):
    return RBF_kernel(inputs, self.centers, self.sigma)

  def get_config(self):
    base_config = super(RBF, self).get.config
    base_config['num_units'] = self.num_units
    return base_config

  def compute_params(self, inputs):
    kmeans = KMeans(n_clusters=self.num_units, random_state=0, copy_x=True).fit(inputs)
    self.centers = kmeans.cluster_centers_
    d_max = ComputeMaxDistance(self.centers, self.centers)
    self.sigma = d_max / (math.sqrt(2 * self.centers.shape[0]))


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import preprocessing
import tensorflow as tf 
from tensorflow import keras
from keras import Input
from keras import layers
from keras.optimizers import RMSprop
import utils
import rbf_layer

# ---------------------------------------------------------------------------- #
#                                   read data                                  #
# ---------------------------------------------------------------------------- #

data = "http://lib.stat.cmu.edu/datasets/boston"
# data = "boston.csv"
raw_df = pd.read_csv(data, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

# ---------------------------------------------------------------------------- #
#                                normalize data                                #
# ---------------------------------------------------------------------------- #

preprocessing.scale(X, copy=False)
preprocessing.scale(y, copy=False)

# ---------------------------------------------------------------------------- #
#                      split data to training and testing                      #
# ---------------------------------------------------------------------------- #

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)

n_train = X_train.shape[0]
hidden_size = int(0.1 * n_train)

In [10]:
rbf_layer_ = RBF(hidden_size)
rbf_layer_.compute_params(X_train)

from keras import layers

model = keras.Sequential()
model.add(Input(shape=(13,)))
model.add(rbf_layer_)
model.add(layers.Dense(8, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))


In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5, batch_size=10)

In [12]:
# print(history.params)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rbf_2 (RBF)                 (None, 37)                482       
                                                                 
 dense (Dense)               (None, 8)                 304       
                                                                 
Total params: 786
Trainable params: 304
Non-trainable params: 482
_________________________________________________________________


In [43]:
import tensorflow as tf 
import numpy as np

a = tf.Variable(np.array([[1., 2., 3.], [4., 5., 6.]]))
b = tf.Variable(np.array([[1., 2., 3.], [4., 3., 4.]]))


def euclideanDistance(x, y):
    dist = tf.sqrt(tf.reduce_sum(tf.square(x - y), 1))
    return dist

d = euclideanDistance(a, b)

print(tf.norm(a-b))

tf.Tensor(2.8284271247461903, shape=(), dtype=float64)
